# Exploring SQLAlchemy Joins a Touch 

First we need to setup our environment to answer the questions from the blog post
* Setting up our ORM objects
* Creating the tables in a SQLite database
* Configuring and initializing a session for us to use for our exploration

In [2]:
from datetime import datetime

from sqlalchemy import (Column, Integer, Numeric, String, DateTime, 
                        ForeignKey, Boolean, create_engine)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref


Base = declarative_base()


class Cookie(Base):
    __tablename__ = 'cookies'

    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12, 2))
    
    def __repr__(self):
        return "Cookie(cookie_name='{self.cookie_name}', " \
                       "cookie_recipe_url='{self.cookie_recipe_url}', " \
                       "cookie_sku='{self.cookie_sku}', " \
                       "quantity={self.quantity}, " \
                       "unit_cost={self.unit_cost})".format(self=self)
    
    
class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)
    
    def __repr__(self):
        return "User(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)
    

class Order(Base):
    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    shipped = Column(Boolean(), default=False)
    
    user =  relationship("User", backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Order(user_id={self.user_id}, " \
                      "shipped={self.shipped})".format(self=self)


class LineItem(Base):
    __tablename__ = 'line_items'
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12, 2))
    
    order = relationship("Order", backref=backref('line_items', order_by=line_item_id))
    cookie = relationship("Cookie", uselist=False)

    def __repr__(self):
        return "LineItems(order_id={self.order_id}, " \
                          "cookie_id={self.cookie_id}, " \
                          "quantity={self.quantity}, " \
                          "extended_cost={self.extended_cost})".format(
                    self=self)    


# Connect do an in memory SQLite database                
engine = create_engine('sqlite:///:memory:')
# Create our tables in that database
Base.metadata.create_all(engine)

# Configure our sessions to use the SQLite database engine
Session = sessionmaker(bind=engine)
# Initialize a session
session = Session()

One of the readers of Essential SQLAlchemy sent me an email with more questions about how .join() works.  In the example below, he wanted to know why join was only required for User, LineItem, and Cookie objects. Why isn't Order required?

In [5]:
query = session.query(Order.order_id, User.username, User.phone,
                      Cookie.cookie_name, LineItem.quantity,
                      LineItem.extended_cost)
query = query.join(User).join(LineItem).join(Cookie)
results = query.filter(User.username == 'cookiemon').all()

To answer that question, lets take a look at the SQL generated by the ORM for our query.

In [6]:
query = session.query(Order.order_id, User.username, User.phone,
                      Cookie.cookie_name, LineItem.quantity,
                      LineItem.extended_cost)
print(query)

SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone, cookies.cookie_name AS cookies_cookie_name, line_items.quantity AS line_items_quantity, line_items.extended_cost AS line_items_extended_cost 
FROM orders, users, cookies, line_items


We can see that the FROM clause contains the Orders, Users, Cookies, and LineItems ORM objects \__tablename\__s for each object in the query. Also, notice the order is based on where they appeared in the SELECT clause. Just like in SQL, we need to define how the tables are related with JOIN clauses. These JOIN clauses need to follow the order of the relationships between the tables. This means we need to make sure that the table to the left of the JOIN clause has a relationship with the table in the .join() statement. This can be a bit confusing when we have chained .join() statements as shown in the first example. The table in the prior .join() statement to the left must have a relationship with the table in the current .join() statement that was being evaluated. Lets look at the SQL generated after all the .join() statements.

In [7]:
query = query.join(User).join(LineItem).join(Cookie)
print(query)

SELECT orders.order_id AS orders_order_id, users.username AS users_username, users.phone AS users_phone, cookies.cookie_name AS cookies_cookie_name, line_items.quantity AS line_items_quantity, line_items.extended_cost AS line_items_extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id


We can see now that the FROM clause contains the JOIN clauses in the order we chained them into the query. So Order is the target of the first JOIN with User, which is why we didn't have to have a .join() for it.